In [9]:
import os 
#import wget
import urllib.request
import requests
import shutil
import fileinput
from pysep import recsec
from pysep.recsec import plotw_rs
from obspy import read, Stream,read_events
from glob import glob
import pandas as pd
from pysep.utils.io import read_sem, read_sem_cartesian
from pysep.utils.io import read_stations
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees
import numpy as np

In [56]:
url_dsm='https://github.com/UT-GlobalSeismology/DSMsynTI-mpi/archive/refs/heads/master.zip'
dsmti_file = urllib.request.urlretrieve(url_dsm,'DSMsynTI-mpi-master.zip')
file = shutil.unpack_archive(dsmti_file[0],'./')


In [28]:
def set_compile_paramters(makefile,dsmtif,maxngrid_r,maxlmax,max_nstation):
    with fileinput.FileInput(makefile, 
                            inplace = True, backup ='.bak') as f:     
        for line in f:
            if "mpif90" in line :
                print(line.replace("mpif90",
                                "mpif90 -std=legacy"), end ='')
            elif "ifort" in line:
                print(line.replace("ifort",
                               "gfortran"), end ='')
            else:
                print(line, end ='')
    with fileinput.FileInput(dsmtif, 
                            inplace = True, backup ='.bak') as f:
        
        for line in f:
            if "parameter ( maxnlay" in line:
                print('        parameter ( maxnlay= '+str(maxngrid_r)+' )', end ='\n')
            elif "parameter ( maxlmax" in line:
                print('        parameter ( maxlmax= '+str(maxlmax)+' )', end ='\n')
            elif "parameter ( maxnr" in line:
                print('        parameter ( maxnr= '+str(max_nstation)+' )', end ='\n')
            else:
                print(line, end ='')

In [22]:
maxngrid_r=88300
maxlmax=80000
max_nstation = 1500
makefile='makefile'

In [57]:
use_own_data=0
dir_path=dsmti_file[0][:-4]
os.chdir(dir_path)
main_file='mpi-tish.f'
os.chdir('tish-mpi')
set_compile_paramters(makefile,main_file,maxngrid_r,maxlmax,max_nstation)
!make
!./tish <examples/test1.inf
os.chdir('../tipsv-mpi')
main_file='mpi-tipsv.f'
set_compile_paramters(makefile,main_file,maxngrid_r,maxlmax,max_nstation)
!make
!./tipsv <examples/test1.inf
os.chdir('../spcsac')
with fileinput.FileInput(makefile, 
                        inplace = True, backup ='.bak') as f:     
    for line in f:
        if "CC =" in line :
            print(line.replace("icx",
                            "gcc"), end ='')
        else:
            print(line, end ='')
!make
os.chdir('example')
if use_own_data==1:
    !cp ../../tish-mpi/examples/*spc .
    !cp ../../tipsv-mpi/examples/*spc .
!../spcsac -l 8

gfortran -O -no-pie -c calmat.f 
gfortran -O -no-pie -c trialf.f 
gfortran -O -no-pie -c others.f 
gfortran -O -no-pie -c dclisb.f 
gfortran -O -no-pie -c dclisb3.f 
gfortran -O -no-pie -c formpi.f 
mpif90 -std=legacy -O -no-pie -c mpi-tish.f -fdec -o mpi-tish.o
mpi-tish.f:149:24:

  141 |         call MPI_BCAST (re, 1, MPI_DOUBLE_PRECISION, 0,
      |                        2
......
  149 |         call MPI_BCAST (np, 1, MPI_INTEGER, 0, MPI_COMM_WORLD, ierr)
      |                        1
mpi-tish.f:155:24:

  141 |         call MPI_BCAST (re, 1, MPI_DOUBLE_PRECISION, 0,
      |                        2
......
  155 |         call MPI_BCAST (vrmin, maxnzone, MPI_DOUBLE_PRECISION, 0,
      |                        1
mpi-tish.f:157:24:

  141 |         call MPI_BCAST (re, 1, MPI_DOUBLE_PRECISION, 0,
      |                        2
......
  157 |         call MPI_BCAST (vrmax, maxnzone, MPI_DOUBLE_PRECISION, 0,
      |                        1
mpi-tish.f:159:24:

  141 |         call 